In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get("https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo2OTE0MDcwNTQzOTI5MTY2OjE3NDU2NzI5MDA6c3BfYXRmOjMwMDQ2ODY3OTg0NTUzMjo6MDo6&url=%2FHP-i5-1235U-Anti-Glare-Micro-Edge-fy5009tu%2Fdp%2FB0D4M66F6R%2Fref%3Dsr_1_1_sspa%3Fcrid%3D1GBCI6Y23VKRI%26dib%3DeyJ2IjoiMSJ9.s11ilc156AsmddiVP65NRYcn36xY1mONbSs5TSkw29jnn2R6H9lcaNudvLtmyjGqvLugoAvlFZZv7ZKZywuRrWWyTKBILBlrTQtzXKYioVTzxizeY3__g10xbdzquq4O0FNG-YiIkK2s_8BzIU3LLgXFAWVTPRCXJIUEMZ9JL9M1V8bJ5ZXdykJyGERYKJc1kioSvnbNGJ0jp-z1DRk-DpAv0nDw4uZsFSCNl0uFkN0.syyf-4QQUTVVcoeswouZlAYNg_-t3ZJdr5yM8sjV-_k%26dib_tag%3Dse%26keywords%3Dlaptops%26qid%3D1745672900%26sprefix%3Dlaptops%252Caps%252C198%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1")

In [36]:
html_data = BeautifulSoup(driver.page_source,"html.parser")

In [37]:
reviewrul = html_data.find('a',{'data-hook':'see-all-reviews-link-foot'})['href']
driver.get("https://www.amazon.in"+reviewrul)

In [8]:
reviews = html_data.find_all("li",{'data-hook':'review'})

In [12]:
url = "https://www.amazon.in/product-reviews/B0CHX1W1XY/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1"

In [31]:
df = pd.read_csv('amazon_mobiles.csv')
purls = df['purls'].tolist()
titles = df['titles'].tolist()


In [ ]:
df = pd.read_csv('amazon_laptops.csv',encoding='latin1')
purls = df['purls'].tolist()
titles = df['titles'].tolist()

k = 0
product_titles = []
reviewTitles = []
reviewTexts = []
ratings = []
for title,purls in zip(titles,purls):
    print(f"{k} Title: ",title)
    driver.get(purls)
    html_data = BeautifulSoup(driver.page_source,"html.parser")
    try:    
        reviewurl = html_data.find('a',{'data-hook':'see-all-reviews-link-foot'})['href']
        reviewurl = "https://www.amazon.in"+reviewurl
        driver.get(reviewurl)
        i = 0
    except:
        reviewurl = None
    while reviewurl is not None:
        i = i+1
        rl = reviewurl + "&pageNumber="+str(i)
        driver.get(rl)
        html_dt = BeautifulSoup(driver.page_source,"html.parser")
        reviews = html_dt.find_all("li",{'data-hook':'review'})
        time.sleep(3)
        for review in reviews:
            try:
                reviewTitle = review.find('a', {'data-hook': 'review-title'}).find_all('span')[-1].text.strip()
                reviewText = review.find('span', {'data-hook': 'review-body'}).span.text.strip()
                rating = review.find('i',{'data-hook':'review-star-rating'}).span.text.strip().replace(" out of 5 stars","")
                rating = float(rating)
                # reviewdate = review.find('span', {'data-hook': 'review-date'}).text.strip().replace("Reviewed in India on ","")
                
                product_titles.append(title)
                reviewTitles.append(reviewTitle)
                reviewTexts.append(reviewText)
                ratings.append(rating)
                # print("-"*20)
            except Exception as e:
                continue
        try:        
            ul = html_dt.find("ul",{'class':'a-pagination'})
            next = ul.find("li",{'class':'a-disabled a-last'})
        except:
            next = "Next"
        if next is not None:
            break
    k += 1

Title:  Apple iPhone 15 Plus (256 GB) - Pink
Title:  Apple iPhone 15 (256 GB) - Pink
Title:  Samsung Galaxy M05 (Mint Green, 4GB RAM, 64 GB Storage) | 50MP Dual Camera | Bigger 6.7" HD+ Display | 5000mAh Battery | 25W Fast Charging | 2 Gen OS Upgrade & 4 Year Security Update | Without Charger
Title:  Redmi A4 5G (Starry Black, 4GB RAM, 128GB Storage) | Global Debut SD 4s Gen 2 | Segment Largest 6.88in 120Hz | 50MP Dual Camera | 18W Fast Charging
Title:  realme NARZO N61 (Voyage Blue,6GB RAM+128GB Storage) 90Hz Eye Comfort Display | IP54 Dust & Water Resistance | 48-Month Fluency | Charger in The Box
Title:  realme GT 6T 5G (Fluid Silver,8GB RAM+256GB Storage) | India's 1st 7+ Gen 3 Flagship Chipset | 1.5M + AnTuTu Score | 5500mAh+120W | The World's Brightest Flagship Display
Title:  realme NARZO N65 5G (Deep Green 6GB RAM, 128GB Storage) India's 1st D6300 5G Chipset | Ultra Slim Design | 120Hz Eye Comfort Display | 50MP AI Camera| Charger in The Box
Title:  Samsung Galaxy M35 5G (Daybr

In [33]:
data = pd.DataFrame({
    'Product Titles': product_titles,
    'Review Titles': reviewTitles,
    'Review Texts': reviewTexts,
    'Ratings': ratings
})

In [34]:
data.to_csv('amazon_mobiles_reviews.csv',index=False)

In [35]:
len(product_titles)

25801